In [3]:
# optimization to save on shuffle space
spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# this needs to be set to a reasonable number on a local machine
spark.conf.set("spark.sql.shuffle.partitions", "16")

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# hack to provide a posix like path in Windows
from pathlib import Path
checkpoint_dir = "file:///" + "/".join(
    (Path("..").resolve() / "data/tmp").parts[1:]
)
print(checkpoint_dir)

# needed by GraphFrames to persist intermediate results to local disk
sc.setCheckpointDir(checkpoint_dir)

file:///Users/acmiy/OneDrive/Documents/cs229/cs229-f19-wiki-forecast/data/tmp


### Recursive bi-partitioning

In [10]:
import pyspark.sql.functions as F
from graphframes import GraphFrame
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.sparse.csgraph import laplacian
import numpy as np
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType
from typing import List


def sparse_matrix_from_edgelist(edges):
    n = edges["src"].max()
    ones = np.ones(len(edges["dst"]))
    return sp.coo_matrix(
        (ones, (edges["src"], edges["dst"])), 
        shape=(n+1, n+1)
    ).tocsr()
    
def fiedler_vector(g):
    L = laplacian(g)
    # ordered by smallest eigenvalue
    _, v = eigsh(L, k=2)
    # fiedler vector is the second smallest eigenvalue
    return v[:,1]


@pandas_udf("id long, value double", PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def compute_fiedler(pdf):
    #relabel?
    g = sparse_matrix_from_edgelist(pdf)
    vec = fiedler_vector(g)
    return pd.DataFrame([{"id": i, "value": v} for i, v in enumerate(vec)])


def induce_graph(graph, relabel=True):
    """Remove extra edges that do not belong to the graph"""
    # small dataframe for reindexing/relabeling
    rank = (
        graph.vertices
        .select("id", F.expr("row_number() over (order by id)").alias("rank"))
        # ensure 0 index for mapping into a scipy.sparse matrix
        .withColumn("rank", F.expr("rank - 1"))
    )
    vertices = graph.vertices.join(rank, on="id", how="inner")
    edges = (
        graph.edges.join(
            vertices.selectExpr("id as src", "rank as relabeled_src"),
            on="src", 
            how="inner"
        ).join(
            vertices.selectExpr("id as dst", "rank as relabeled_dst"), 
            on="dst", 
            how="inner"
        )
    )

    if relabel:
        vertices = (
            vertices
            .withColumn("original_id", F.col("id"))
            .withColumn("id", F.col("rank"))
        )
        edges = edges.selectExpr("relabeled_src as src", "relabeled_dst as dst")
    vertices = vertices.drop("rank")
    edges = edges.drop("relabeled_src").drop("relabeled_dst")

    return GraphFrame(vertices, edges)


def sample_graph(pages, pagelinks, sampling_ratio, relabel=True):
    vertices = pages.sample(sampling_ratio)
    edges = pagelinks.selectExpr("from as src", "dest as dst")
    return induce_graph(GraphFrame(vertices, edges), relabel)


def recursive_bipartition(graph: GraphFrame, max_iter: int = 2) -> GraphFrame:
    def undo_relabel(vertices):
        return vertices.withColumn("id", F.col("original_id")).drop("original_id")

    """
    Assumes the input graph has been relabeled, which is required for performance of
    scipy.linalg.eigsh.
    """
    def bipartition(graph: GraphFrame, partitions: List[str] = [], iteration: int = 0):
        
        partition = f"sign_{iteration}"
        fiedler_value = f"fiedler_{iteration}"
        fiedler = (
            graph
            .edges
            # necessary for collecting all edges to a single worker
            .withColumn("part", F.lit(True))
            .groupBy("part")
            .apply(compute_fiedler)
            .withColumn(partition, F.expr("value >= 0").astype("boolean"))
            .selectExpr("id", f"value as {fiedler_value}", partition)
        )
        
        # NOTE: assumes relabeling, reverse the relabeling process
        vertices = graph.vertices.join(fiedler, on="id", how="left")
        parted_graph = GraphFrame(vertices, graph.edges)
        
        if iteration == max_iter:
            return undo_relabel(parted_graph.vertices)
        else:
            positive_vertices = bipartition(
                induce_graph(
                    GraphFrame(
                        parted_graph.vertices.where(f"{partition}"),
                        parted_graph.edges,
                    ),
                    relabel=True,
                ),
                partitions + [partition],
                iteration + 1,
            )
            negative_vertices = bipartition(
                induce_graph(
                    GraphFrame(
                        parted_graph.vertices.where(f"NOT {partition}"),
                        parted_graph.edges,
                    ),
                    relabel=True,
                ),
                partitions + [partition],
                iteration + 1,
            )
            # reusing the index, should this be saved?
            return undo_relabel(
                positive_vertices
                .union(negative_vertices)
                .join(vertices.select("id", "original_id"), on="id", how="inner")
            )

    return bipartition(induce_graph(graph, relabel=True))

In [11]:
pages_path = "../data/enwiki/pages"
pagelinks_path = "../data/enwiki/pagelinks"

pages = spark.read.parquet(pages_path)
pagelinks = spark.read.parquet(pagelinks_path)

In [12]:
g = sample_graph(pages, pagelinks, 0.05)
g.cache()
result = recursive_bipartition(g, max_iter=3)
signs = [c for c in result.columns if c.startswith("sign_")]
%time result.groupBy(signs).count().show()

+------+------+------+------+-----+
|sign_0|sign_1|sign_2|sign_3|count|
+------+------+------+------+-----+
|  true| false|  true| false|19949|
|  true|  true|  true|  true|18782|
|  true|  true| false| false|18339|
| false|  true|  true| false|16796|
| false| false|  true|  true|18214|
| false|  true| false|  true|17170|
|  true|  true| false|  true|18783|
| false| false|  true| false|18222|
|  true|  true| false|  null|   13|
|  true|  true|  true|  null|   12|
|  true| false|  true|  null|   13|
| false|  true| false| false|16911|
|  true|  true|  true| false|18881|
|  true| false|  true|  true|19677|
| false|  true|  true|  true|16686|
|  true| false| false|  true|19879|
| false| false|  true|  null|    7|
|  true| false| false|  null|    2|
| false|  true|  true|  null|    3|
|  true| false| false| false|20352|
+------+------+------+------+-----+
only showing top 20 rows

Wall time: 25min 9s


In [13]:
g = sample_graph(pages, pagelinks, 0.5)
g.cache()
result = recursive_bipartition(g, max_iter=4)
signs = [c for c in result.columns if c.startswith("sign_")]
%time result.groupBy(signs).count().show()

Py4JJavaError: An error occurred while calling o4076.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: makeCopy, tree:
Project [id#833218, (rank#833211 - 1) AS rank#833215]
+- Filter (isnotnull(id#833218) && isnotnull((rank#833211 - 1)))
   +- Window [row_number() windowspecdefinition(id#833218 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#833211], [id#833218 ASC NULLS FIRST]
      +- *(788811) Sort [id#833218 ASC NULLS FIRST], false, 0
         +- Exchange SinglePartition
            +- InMemoryTableScan [id#833218]
                  +- InMemoryRelation [id#833218, title#45, is_redirect#46, is_new#47, original_id#833149], StorageLevel(disk, memory, deserialized, 1 replicas)
                        +- *(7) Project [rank#833122 AS id#833156, title#45, is_redirect#46, is_new#47, id#44 AS original_id#833149]
                           +- *(7) SortMergeJoin [id#44], [id#833125], Inner
                             